## Prereq

In [1]:
from dotenv import load_dotenv
from pathlib import Path
import os

env_path = Path('.') / '.env'
load_dotenv(env_path)

True

In [2]:
import os
# Read OpenAI API key from environment for safety.
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise EnvironmentError('Please set the OPENAI_API_KEY environment variable before running this notebook')
print('OPENAI_API_KEY found in environment, proceeding...')

OPENAI_API_KEY found in environment, proceeding...


## Load

In [3]:
from langchain_community.document_loaders import TextLoader

# Use the CSV in the project data/clean directory (relative to analysis/)
loader = TextLoader("../data/process/character_lines.csv")
documents = loader.load()

## Embed

In [4]:
import pandas as pd
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import os

csv_path = "../data/clean/train.csv"
df = pd.read_csv(csv_path)

# Detect likely columns
possible_char_cols = ["character"]
possible_text_cols = ["lines"]

char_col = next((c for c in possible_char_cols if c in df.columns), None)
text_col = next((c for c in possible_text_cols if c in df.columns), None)

# fallback: if only two columns, assume first is character, second is text
if char_col is None or text_col is None:
    cols = list(df.columns)
    if len(cols) >= 2:
        char_col = char_col or cols[0]
        text_col = text_col or cols[1]
    else:
        # assume single-column CSV, each row may be "CHARACTER: line"
        char_col = None
        text_col = cols[0]

docs = []
for idx, row in df.iterrows():
    raw_text = str(row[text_col]) if text_col in row else str(row[0])
    if not raw_text or raw_text.strip() == "":
        continue
    metadata = {"source": os.path.basename(csv_path), "row": int(idx)}
    if char_col and char_col in row:
        metadata["character"] = str(row[char_col])
    else:
        # try to parse "CHARACTER: text" pattern
        if ":" in raw_text:
            maybe_char, maybe_line = raw_text.split(":", 1)
            metadata["character"] = maybe_char.strip()
            raw_text = maybe_line.strip()
    docs.append(Document(page_content=raw_text, metadata=metadata))

# Create embeddings and vector store
embeddings = OpenAIEmbeddings()  # needs OPENAI_API_KEY env var (not printed)
vectordb = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_lines")
vectordb.persist()

# Example query
results = vectordb.similarity_search("Find lines where Odysseus expresses longing", k=5)
for r in results:
    print(r.page_content, r.metadata)

/tmp/ipykernel_26328/2086565278.py:45: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()  # needs OPENAI_API_KEY env var (not printed)
/tmp/ipykernel_26328/2086565278.py:47: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()
/tmp/ipykernel_26328/2086565278.py:47: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


Odysseus {'source': 'train.csv', 'row': 24, 'character': 'Suitors'}
Odysseus {'source': 'train.csv', 'row': 25, 'character': 'Odysseus'}
Odysseus {'character': 'Suitors', 'row': 24, 'source': 'train.csv'}
Odysseus {'character': 'Odysseus', 'row': 25, 'source': 'train.csv'}
Just keep your eyes open

Just keep your eyes open

Just keep your eyes open

Just keep your eyes open

Wake up!

Wake up, Odysseus, they're opening the bag

Wake up! {'character': 'Penelope', 'row': 31, 'source': 'train.csv'}


In [11]:
retriever = vectordb.as_retriever()

In [15]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model_name="gpt-4.1")
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

query = "which character in epic would say: Wait  Stop this, please  NO  Aren't you tired, Poseidon?  It's been 8 years, how long will this go?  We're both hurting from losses  So why not leave this here and just go home?"
response = qa_chain.invoke({"query": query})
print(response["result"])

The character in the epic most likely to say these words is **Odysseus**.

**Reasoning:**  
- The lines express frustration, exhaustion, and a plea to **Poseidon**, the god who relentlessly persecutes Odysseus after he blinds the Cyclops Polyphemus (Poseidon's son).
- Odysseus is famously kept from returning home for many years (ten years after the Trojan War), largely due to Poseidon’s wrath.
- The tone ("It's been 8 years, how long will this go?") fits Odysseus' situation, as he is desperate to return to Ithaca and wants Poseidon’s punishment to end.
- The sense of mutual suffering and loss also fits Odysseus’ perspective, as he has lost crew and time, and Poseidon has lost his son’s eye.

**Conclusion:**  
**Odysseus** would be the character in the epic most likely to plead with Poseidon in this manner.


In [ ]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma # Or FAISS or others
from langchain.schema import Document
from sentence_transformers import CrossEncoder

# Params
FOLDER_PATH = "../data_new/speakertext_train"  # replace with your folder path
QUERY = "Your query goes here"
CROSS_ENCODER_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"
CHROMA_DB_PATH = "langchain_db"

# Step 1: Load text files
documents = []
for fname in os.listdir(FOLDER_PATH):
    if fname.endswith(".txt"):
        with open(os.path.join(FOLDER_PATH, fname), "r", encoding="utf-8") as f:
            doc_text = f.read()
        documents.append((fname, doc_text))

# Step 2: Score relevance with cross encoder
cross_encoder = CrossEncoder(CROSS_ENCODER_MODEL)
scores = cross_encoder.predict([(QUERY, doc[1]) for doc in documents])

# Optional: Filter based on a score threshold if desired
RELEVANCE_THRESHOLD = 0.0  # update as needed
selected_documents = [
    Document(page_content=doc[1], metadata={"filename": doc[0], "score": float(score)})
    for doc, score in zip(documents, scores)
    if score >= RELEVANCE_THRESHOLD
]

# Step 3: Add to vector database using LangChain
embeddings = HuggingFaceEmbeddings()
vectordb = Chroma(persist_directory=CHROMA_DB_PATH, embedding_function=embeddings)
vectordb.add_documents(selected_documents)
vectordb.persist()

print(f"Added {len(selected_documents)} relevant documents to the database.")